In [32]:
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from pyod.models.iforest import IForest
import sklearn.ensemble as ensemble
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from sklearn.neural_network import MLPRegressor
import sklearn.linear_model as linear_model
from sklearn import tree,linear_model,svm,neighbors,ensemble  
from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.svm import SVR
import warnings
import statistics
import warnings
warnings.filterwarnings('ignore')

In [33]:
# pre-processing

train_x = pd.read_csv("X_train.csv")
train_y = pd.read_csv("y_train.csv")
test_x = pd.read_csv("X_test.csv")

# pre-selection
train_x_imputed = train_x.fillna(train_x.median())
test_x_imputed = test_x.fillna(test_x.median())


In [34]:
combined_x = pd.concat([train_x_imputed,test_x_imputed], axis = 0)
combined_x

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,10295.013382,1.144294e+06,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1,1.0,11.512994,832442.898114,20585.524817,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2,2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3,3.0,11.642076,832442.860041,20585.524817,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,99802.127899,102746.516920
4,4.0,10.407121,832442.831424,20585.557007,1047.985497,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,...,1.031009e+06,-1025.223865,18348.460040,10.554260,10.114370,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,771.0,8.643008,832442.901284,20585.516033,1049.164447,9.634039e+05,11.763651,597900.382191,12498.079536,9.646803e+05,...,1.036885e+06,-1096.877578,16780.272363,10.494413,10.985756,1482.074684,11.709038,1.007811e+06,98257.691777,107061.654854
772,772.0,9.639356,832442.864233,20585.544425,1043.984958,1.147107e+06,11.144385,597900.460932,10499.010672,8.629803e+05,...,1.068757e+06,-942.034551,15097.913035,10.564327,10.691379,762.002833,9.960825,1.030335e+06,110688.236655,100730.307869
773,773.0,10.091882,832442.904383,20585.515217,1017.114468,1.040988e+06,9.840801,597900.441911,11192.048851,9.575331e+05,...,1.061697e+06,-989.876896,14143.508919,11.002609,11.177008,1047.546785,9.023443,1.045739e+06,103970.270645,108906.208766
774,774.0,11.154426,832442.875082,20585.496750,1059.286367,9.187398e+05,9.500992,597900.388416,10931.010976,8.592437e+05,...,1.002105e+06,-848.755303,10119.058965,10.242841,8.894702,1044.042333,10.634034,1.071376e+06,92897.930814,108439.186933


In [35]:
combined_x = combined_x.drop('id',axis = 1)

In [36]:
np_combined_x = combined_x.to_numpy()
np_combined_x.shape

(1988, 832)

In [37]:
np_combined_x += 1e-7
x_var = np.var(np_combined_x/np.mean(np_combined_x,axis = 0), axis = 0)
x_index = x_var > 1e-10
np_combined_x -= 1e-7
np_combined_preselected = np_combined_x[:,x_index]

In [38]:
preselected_standarized = StandardScaler().fit_transform(np_combined_preselected)

In [39]:
y_dropped = train_y.drop('id',axis = 1).to_numpy()
y_dropped.shape

(1212, 1)

In [40]:
y_dropped

array([[71.],
       [73.],
       [66.],
       ...,
       [82.],
       [76.],
       [81.]])

In [41]:
x_train_preselected = preselected_standarized[:len(train_x_imputed)]
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
kbest= SelectKBest(score_func=f_classif, k=150)
kbest.fit(x_train_preselected, y_dropped)
x_selected = kbest.transform(x_train_preselected)

In [42]:
index = np.arange(sum(x_index))
index = kbest.transform(index.reshape((1, -1)))
kbest_index_150 = index[0]

In [43]:
#fill missing part with random forest
combined = pd.concat([train_x,test_x], axis = 0)
combined = combined.drop('id',axis = 1)
missing_index = np.isnan(combined.to_numpy())

missing_preselect = missing_index[:,x_index]
missing_kbest = missing_preselect[:,kbest_index_150]

x_train_standarized = preselected_standarized[:len(train_x)]
x_test_standarzied = preselected_standarized[len(train_x):]
standarzied_kbest = preselected_standarized[:,kbest_index_150]

In [44]:
for i in range(150):
            data_train =standarzied_kbest[missing_kbest[:, i] == 0]
            data_test = standarzied_kbest[missing_kbest[:, i] == 1]
            x = data_train[:, np.concatenate((np.arange(i), np.arange(i + 1, 150)))]
            y = data_train[:, i]
            x_test = data_test[:, np.concatenate((np.arange(i), np.arange(i + 1, 150)))]
            random_forest = ensemble.RandomForestRegressor(n_estimators=20)
            random_forest.fit(x, y)
            standarzied_kbest[missing_kbest[:, i] == 1, i] = random_forest.predict(x_test)

In [45]:
x_train_filled = standarzied_kbest[:len(train_x)]
x_test_filled = standarzied_kbest[len(train_x):]
np.save('x_filled_150.npy',x_train_filled)
np.save('x_test_filled_150.npy',x_test_filled)

## utilize a specific random forest generated dataset to keep everything the same and save time (random forest imputation results differ from time to time)

In [51]:
x_train_filled = np.load('x_filled_random_forest_150.npy')
pd.DataFrame(x_train_filled)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.067006,-1.293425,-1.403565,-0.752193,0.785924,-0.571447,-0.276164,0.607955,-0.653109,-1.011140,...,-0.720488,-1.456983,0.224129,-0.462247,-0.045539,-1.852063,-0.408639,0.606301,0.147695,-0.558470
1,-1.469999,0.773029,-1.160758,-2.384768,-0.234770,-1.298215,-3.968961,-3.920331,-0.852204,-3.227427,...,-1.452378,-0.553620,-4.604982,-0.912848,-2.907321,2.005624,-2.809605,1.324355,0.599145,-1.296270
2,0.248151,0.095817,-0.331348,1.710010,-0.852011,-1.140590,0.474671,0.675400,-0.452269,1.017093,...,1.285088,0.710090,2.336358,-0.527738,0.244477,-0.498429,1.312145,1.309645,-0.193105,-1.091796
3,2.081409,1.765467,2.266001,2.262642,1.014391,1.462275,0.594757,1.729658,2.066897,2.069848,...,0.736901,2.166233,2.487331,0.788261,1.693366,1.066877,2.519459,-1.441200,-0.886959,1.488473
4,1.666833,-0.402217,1.825465,0.093205,0.759559,2.133440,0.788909,1.071193,1.388511,0.794569,...,1.026589,0.735781,-0.020892,0.978279,0.232978,0.450529,0.284147,-2.170518,-0.976263,2.147437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,1.522846,0.797356,0.479127,0.679592,-0.289678,1.848432,1.064154,0.848504,1.669260,1.266909,...,0.546970,1.935008,1.150798,2.118225,0.416713,0.934763,1.878352,-1.890695,-0.805464,1.858508
1208,0.316553,2.376054,0.722423,1.526327,1.003326,1.036352,0.139742,0.310240,0.790403,1.288308,...,1.061305,0.392017,1.502281,1.164241,1.311924,0.570094,1.394910,-0.833109,-1.558192,1.115298
1209,-1.979360,-0.823153,-1.432517,-0.701362,-1.439913,-0.694137,-0.296712,-1.520557,-1.350726,-1.728302,...,-1.730668,-0.302337,-1.124100,-0.764278,-1.199191,-0.597893,-1.367733,0.231197,2.621635,-0.837492
1210,1.816189,1.358641,2.106213,0.851736,-0.659798,0.658404,0.162908,0.657179,1.562514,0.889259,...,0.483502,1.372113,-0.063667,3.412383,0.555923,0.960407,0.427011,-0.479488,-1.230829,0.727154


In [52]:
x_test_filled = np.load('x_test_filled_random_forest_150.npy')
pd.DataFrame(x_test_filled)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0.283004,0.393895,-0.701723,-0.019336,1.143775,-0.990787,0.593992,1.081655,1.019764,-0.173057,...,-0.309378,0.141915,-1.108366,1.356631,0.378535,0.254133,0.358797,1.276945,-0.824324,-0.903857
1,-0.026285,0.541333,-0.061505,0.214694,0.267189,0.938926,0.826456,0.651093,0.334588,0.176864,...,1.704390,0.058003,-0.036650,0.651784,0.850816,-0.672572,0.597514,-1.179471,0.666992,0.878960
2,-0.636262,-0.248033,0.907850,-0.420359,1.398358,0.072832,-1.212623,0.891306,-0.285484,-0.590058,...,0.262110,-0.800316,-0.646663,-0.239720,-0.609480,-0.024626,-0.493969,-0.095786,0.077080,0.072913
3,0.183624,1.158711,1.134022,0.101583,-0.224763,0.481676,-0.132390,-0.098889,1.199175,-0.222020,...,-0.527875,1.944100,-0.378860,1.555416,-0.384747,0.344415,0.252019,-0.451370,-0.406746,0.502557
4,0.067833,0.779854,0.318337,1.177823,0.102110,0.318433,1.286779,0.362424,-0.034634,-0.405409,...,-0.294231,-0.526184,0.514367,-0.034114,0.637459,-0.542146,-0.365107,-0.144378,-1.029844,0.381924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,1.406265,1.835712,1.098948,1.450787,0.507072,1.396482,1.405842,1.622259,1.515080,1.311798,...,1.459262,1.864717,0.587221,1.324934,1.952536,1.882977,0.970651,-1.253905,-1.450912,1.459961
772,0.090830,0.169374,1.595927,-1.765820,0.085927,0.706458,-1.212707,-0.068065,0.955148,-0.326406,...,-0.244515,0.683263,-0.964681,0.844034,-0.412458,0.352048,-0.519692,-0.693807,-0.425199,0.722433
773,0.546865,0.751724,-0.051563,-0.456691,-0.780042,0.256389,-1.287434,-0.495091,0.512019,-1.003578,...,-1.136913,0.532488,-0.382298,-0.581733,-0.756550,0.171576,-1.220467,-0.132562,-0.742117,0.304032
774,0.375096,0.779996,0.036200,-0.726173,0.441715,-1.494393,0.287215,-0.109627,0.421050,-0.084022,...,0.207047,0.316072,0.267792,0.509293,0.310094,0.655086,-0.668906,1.624490,0.192702,-1.460243


In [46]:
classifier = IForest(contamination=0.04, random_state=42)
classifier.fit(x_train_filled)
outliers_detect = classifier.predict(x_train_filled)
id_outlier = [i for i in range(0, 1212) if outliers_detect[i] == 1]
len(id_outlier)

49

In [47]:
x_train_clean = np.delete(x_train_filled, id_outlier, axis=0)

In [48]:
y_train_clean = np.delete(y_dropped, id_outlier, axis=0)

In [49]:
print(x_train_clean.shape)
print(y_train_clean.shape)

(1163, 150)
(1163, 1)


In [50]:
lasso_slector = linear_model.Lasso(alpha=0.01)
selector = SelectFromModel(lasso_slector.fit(x_train_clean, y_train_clean),prefit = True)

In [51]:
x_select = selector.transform(x_train_clean)
x_test_select = selector.transform(x_test_filled)

In [52]:
print(x_select.shape)
print(x_test_select.shape)

(1163, 117)
(776, 117)


In [53]:
y_select = y_train_clean
y_select.shape

(1163, 1)

In [54]:
# stacking model

model_names = []
models = []

model_LinearRegression = linear_model.LinearRegression()
model_names.append("Linear Regression")
models.append(model_LinearRegression)

    # 2
model_SVR = SVR(kernel = 'rbf',C= 35, epsilon= 0.008, gamma=0.008)
model_names.append("Support Vector Machine Regression")
models.append(model_SVR)

   # 3
model_KNeighborsRegressor = neighbors.KNeighborsRegressor(leaf_size=11, p=2,n_neighbors = 4,algorithm='brute',weights='distance')
model_names.append("K-Nearest Neighbor Regression")
models.append(model_KNeighborsRegressor)

    # 4
# model_RandomForestRegressor = ensemble.RandomForestRegressor(n_estimators=20)
# model_names.append("Random Forest Regression")
# models.append(model_RandomForestRegressor)

# adaboost
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(base_estimator=tree.DecisionTreeRegressor(), random_state=42, n_estimators=2000,loss='linear',learning_rate = 0.3)
#model_AdaBoostRegressor = ensemble.AdaBoostRegressor(base_estimator=ExtraTreeRegressor(ccp_alpha=0.0,max_features='auto', 
#                                                                                       min_impurity_decrease=0.0,
#                                                                                       min_samples_leaf=1, 
#                                                                                       min_samples_split=2,
#                                                                                       min_weight_fraction_leaf=0.0, 
#                                                                                       random_state=666,splitter='random'), 
#                                                     random_state=42, n_estimators=1500,loss='linear',learning_rate = 0.4)
model_names.append("AdaBoost Regression")
models.append(model_AdaBoostRegressor)

# gradient boosting
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                            max_depth=4, max_features='sqrt',
                                            min_samples_leaf=15, min_samples_split=10,
                                            loss='huber', random_state =5)
model_names.append("Gradient Boosting Regression")
models.append(model_GradientBoostingRegressor)

     # 8
model_ExtraTreeRegressor = ensemble.ExtraTreesRegressor(n_jobs=1, max_depth=15, n_estimators=195, random_state=0, min_samples_split=3, max_features=None)
model_names.append("ExtraTree Regression")
models.append(model_ExtraTreeRegressor)

       # 9
model_XGBoostRegressor = xgb.XGBRegressor(learning_rate=0.01,n_estimators=3000,max_depth=4,min_child_weight=0,gamma=0.6,
                                            subsample=0.7,colsample_bytree=0.7,nthread=-1,scale_pos_weight=1,seed=1,reg_alpha=0.00006,random_state=1)
model_names.append("XGBoost Regression")
models.append(model_XGBoostRegressor)

model_lasso = linear_model.LassoCV(cv=5, random_state=100)
model_names.append("Lasso Regression")
models.append(model_lasso)

model_ridge = linear_model.RidgeCV(cv=5)

model_KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
model_names.append("Kernel Ridge Regression")
models.append(model_KRR)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_names.append('LGBM Regressor')
models.append(model_lgb)

model_mlp = MLPRegressor(activation = 'identity',learning_rate_init = 0.0001, batch_size = 24,
                    alpha = 0.001, random_state=1, max_iter=500)
model_names.append('MLP Regressor')
models.append(model_mlp)

kernel = 8*Matern(nu=1.5)
gpr = GaussianProcessRegressor(kernel = kernel, alpha=5e-9, optimizer='fmin_l_bfgs_b', normalize_y=True,random_state=666)
model_names.append('Gaussian')
models.append(gpr)


In [55]:
est = list(zip(model_names,models))

stacked = StackingRegressor(estimators=est,final_estimator = model_ridge)

score_func = r2_score
kf = KFold(n_splits=5, shuffle=True)

score = []
for train_idx, test_idx in kf.split(x_select):
    x_train = x_select[train_idx]
    y_train = y_select[train_idx]
    x_test = x_select[test_idx]
    y_test = y_select[test_idx]
    stacked.fit(x_train,y_train)
    s = stacked.score(x_test,y_test)
    print('score: ', s)
    score.append(s)

score


[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11

[0.6904116445645695,
 0.6478741955768405,
 0.7153178604124563,
 0.7276531442404763,
 0.6948488276954488]

In [56]:
sum(score)/5

0.6952211344979583

In [323]:
stacked.fit(x_select,y_select)
stacked_pred = stacked.predict(x_test_select)

model_XGBoostRegressor.fit(x_select,y_select)
xgb_pred = model_XGBoostRegressor.predict(x_test_select)

#model_lgb.fit(x_select,y_select)
#lgb_pred = model_lgb.predict(x_test_select)

gpr.fit(x_select,y_select)
gpr_pred = gpr.predict(x_test_select)

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11

In [354]:
final_pred_with_gaussian = pd.DataFrame(stacked_pred).to_numpy()*0.6+pd.DataFrame(xgb_pred).to_numpy()*0.15+gpr_pred*0.25
pd.DataFrame(final_pred_with_gaussian).to_csv('submission_stack(ridge)_gaussian_weighted_0.25.csv')